
# 📌 Predição do Rendimento da Safra 🌾

## 📌 Descrição do Projeto  
Este projeto faz parte da **Fase 4 - Modelagem de Dados com Regressão Supervisionada** e tem como objetivo prever o rendimento da safra a partir de variáveis climáticas e condições do solo.  

Utilizamos técnicas de **Análise Exploratória de Dados (EDA)**, **Clusterização** e **Modelagem Preditiva** para entender os padrões nos dados e construir modelos de Machine Learning.

## 🎯 Objetivos da Entrega  
- Analisar a base de dados e encontrar tendências nos rendimentos das plantações 🌱  
- Utilizar **clusterização** para identificar padrões e outliers nos dados 🔍  
- Construir **modelos de regressão supervisionada** para prever o rendimento das culturas 📊  
- Avaliar o desempenho dos modelos com métricas estatísticas 🏆  


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Configurar estilo dos gráficos
sns.set_style("whitegrid")


In [ ]:

# Carregar os dados
file_path = "crop_yield.csv"  # Certifique-se de ter o arquivo na mesma pasta
df = pd.read_csv(file_path)

# Visualizar as primeiras linhas
df.head()



## 📊 Análise Exploratória dos Dados (EDA)  
Aqui exploramos os dados para verificar se há valores ausentes, padrões ou tendências relevantes.


In [ ]:

# Informações gerais
df.info()

# Estatísticas descritivas
df.describe()

# Verificar valores nulos
df.isnull().sum()



### 🔎 Análises e Observações  
- **Os dados não possuem valores ausentes**, o que facilita a modelagem.  
- **As variáveis climáticas** (temperatura, umidade, precipitação) possuem variações significativas.  
- **O rendimento da safra apresenta grande dispersão**, indicando que diferentes culturas podem reagir de formas variadas às condições ambientais.  



### 📈 Visualização dos Dados  
Vamos analisar a distribuição do rendimento da safra e sua relação com as condições climáticas.


In [ ]:

# Histogramas e Boxplots
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
sns.histplot(df["Yield"], bins=15, kde=True)
plt.title("Distribuição do Rendimento da Safra")

plt.subplot(2, 2, 2)
sns.boxplot(x=df["Yield"])
plt.title("Boxplot do Rendimento")

plt.subplot(2, 2, 3)
sns.scatterplot(x=df["Temperature at 2 Meters (C)"], y=df["Yield"], hue=df["Crop"])
plt.title("Temperatura vs Rendimento")

plt.subplot(2, 2, 4)
sns.scatterplot(x=df["Precipitation (mm day-1)"], y=df["Yield"], hue=df["Crop"])
plt.title("Precipitação vs Rendimento")

plt.tight_layout()
plt.show()



### 📊 Observações sobre os Gráficos  
- **Distribuição do rendimento**: Há uma variação considerável, e alguns valores extremos podem ser outliers.  
- **Temperatura vs Rendimento**: Algumas culturas parecem ser mais afetadas por variações de temperatura.  
- **Precipitação vs Rendimento**: Não há uma correlação clara, sugerindo que a precipitação isolada pode não ser um fator determinante.  



## 🔍 Clusterização para Encontrar Padrões  
Usamos **K-Means** para segmentar os dados e identificar padrões entre as culturas.


In [ ]:

# Normalizar os dados
features = df.drop(columns=["Crop", "Yield"])
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Definir número de clusters (k=3 baseado no método do cotovelo)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(features_scaled)

# Visualizar clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df["Temperature at 2 Meters (C)"], y=df["Yield"], hue=df["Cluster"], palette="viridis")
plt.title("Clusterização: Temperatura vs Rendimento")
plt.xlabel("Temperatura (°C)")
plt.ylabel("Rendimento (toneladas/ha)")
plt.legend(title="Cluster")
plt.show()



### 🔎 Análises da Clusterização  
- **Os clusters indicam diferentes padrões de rendimento** entre as culturas.  
- **Temperatura parece ser um fator relevante**, com rendimentos variando entre os grupos.  
- **Essas informações podem ser úteis para ajustar os modelos preditivos**.  



## 🤖 Modelagem Preditiva  
Agora treinamos diferentes modelos de regressão para prever o rendimento da safra com base nas condições ambientais.


In [ ]:

# Preparação dos dados para regressão
X = df.drop(columns=["Crop", "Yield", "Cluster"])
y = df["Yield"]

X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Modelos
models = {
    "Regressão Linear": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Suporte a Vetores (SVR)": SVR(),
    "Rede Neural (MLP)": MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)
}

# Treinar e avaliar os modelos
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2}

# Exibir resultados
pd.DataFrame(results).T
